<a href="https://colab.research.google.com/github/asuzukosi/ai-experiments/blob/main/src/react_agent_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Setup

In [1]:
from google.colab import userdata


import os
os.environ["WANDB_API_KEY"] = "43f707a894e9e29362dda1b3706450e2b074e84b"
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_KEY')

In [2]:
!git clone https://github.com/run-llama/llama_index.git

Cloning into 'llama_index'...
remote: Enumerating objects: 49331, done.
remote: Counting objects: 100% (15761/15761), done.
remote: Compressing objects: 100% (1811/1811), done.
remote: Total 49331 (delta 14356), reused 14109 (delta 13946), pack-reused 33570
Receiving objects: 100% (49331/49331), 106.71 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (35889/35889), done.


In [3]:
%cd llama_index/docs/examples/finetuning/react_agent

/content/llama_index/docs/examples/finetuning/react_agent


In [4]:
!pip install llama_index openai wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstall

In [5]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 2.4 MB/s eta 0:00:00


In [7]:
import os
import glob
import json
import random
import tiktoken
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from collections import defaultdict
from tenacity import retry, stop_after_attempt, wait_fixed
import openai
openai.api_key=os.environ["OPENAI_API_KEY"]

# Fine-tuning a gpt-3.5 ReAct Agent on Better Chain of Thought

In this guide, we fine-tune a ReAct Agent powered by gpt-3.5-turbo to perform better chain-of-thought prompting over financial statements.

We do this in the following steps:
1. Setup LlamaIndex query engine tools over Uber 10Q filings.
2. Use our dataset generator to generate a training/evaluation question dataset over a sample 10Q filing. Add complex variations to each question to account for multiple quarters (these complex questions help to induce chain-of-thought prompting).
3. Feed these questions through a GPT-4 ReAct Agent. Log inputs/outputs as a dataset to fine-tune over.
4. Call OpenAI fine-tuning endpoints to fine-tune gpt-3.5-turbo on this dataset.
5. Run qualitative evaluation: show that the fine-tuned model performs better in chain-of-thought prompting than the base model.

#### Note
Each execution of an agent can involve multiple LLM calls through the ReAct chain-of-thought loop. The prompt inputs/output pair for each LLM call is logged as an individual datapoint in the training dataset, in the chat message format.

A big TODO here is to add more quantitative metrics for better evaluation.

## Setup Data + Build Query Engine Tools

In this section, we load in 3 Uber 10Q fiings (March, June, September). We also setup a standard vector index over each document. This gives the agent the tools to do vector search within any given document.

In [9]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    ServiceContext,
    load_index_from_storage,
)
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata

In [10]:
import llama_index
from llama_index import set_global_handler

set_global_handler("wandb", run_args={"project": "llama_index-react-agent-finetuning"})
wandb_callback = llama_index.global_handler

wandb: Streaming LlamaIndex events to W&B at https://wandb.ai/keloasuzu/llama_index-react-agent-finetuning/runs/98v5n6hm
wandb: `WandbCallbackHandler` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `llamaindex`.


In [11]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-0613")
# llm = OpenAI(temperature=0, model="gpt-4-0613")
service_context = ServiceContext.from_defaults(llm=llm)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo-0613", temperature=0.3)
)
gpt4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4-0613", temperature=0.3)
)

In [12]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/march"
    )
    march_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/june"
    )
    june_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/sept"
    )
    sept_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [13]:
if not index_loaded:
    # load data
    march_docs = SimpleDirectoryReader(
        input_files=["../../data/10q/uber_10q_march_2022.pdf"]
    ).load_data()
    june_docs = SimpleDirectoryReader(
        input_files=["../../data/10q/uber_10q_june_2022.pdf"]
    ).load_data()
    sept_docs = SimpleDirectoryReader(
        input_files=["../../data/10q/uber_10q_sept_2022.pdf"]
    ).load_data()

    # build index
    march_index = VectorStoreIndex.from_documents(
        march_docs, service_context=service_context
    )
    june_index = VectorStoreIndex.from_documents(
        june_docs, service_context=service_context
    )
    sept_index = VectorStoreIndex.from_documents(
        sept_docs, service_context=service_context
    )

    # persist index
    march_index.storage_context.persist(persist_dir="./storage/march")
    june_index.storage_context.persist(persist_dir="./storage/june")
    sept_index.storage_context.persist(persist_dir="./storage/sept")

wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.


In [14]:
march_engine = march_index.as_query_engine(
    similarity_top_k=3, service_context=service_context
)
june_engine = june_index.as_query_engine(
    similarity_top_k=3, service_context=service_context
)
sept_engine = sept_index.as_query_engine(
    similarity_top_k=3, service_context=service_context
)

In [15]:
from llama_index.tools.query_engine import QueryEngineTool


query_tool_sept = QueryEngineTool.from_defaults(
    query_engine=sept_engine,
    name="sept_2022",
    description=(
        f"Provides information about Uber quarterly financials ending"
        f" September 2022"
    ),
)
query_tool_june = QueryEngineTool.from_defaults(
    query_engine=june_engine,
    name="june_2022",
    description=(
        f"Provides information about Uber quarterly financials ending June"
        f" 2022"
    ),
)
query_tool_march = QueryEngineTool.from_defaults(
    query_engine=march_engine,
    name="march_2022",
    description=(
        f"Provides information about Uber quarterly financials ending March"
        f" 2022"
    ),
)

query_engine_tools = [query_tool_march, query_tool_june, query_tool_sept]

## Setup Base ReAct Agent (gpt-3.5-turbo)

Here we define the baseline ReAct agent over our data, on top of gpt-3.5-turbo.

We run some example queries, and show that the ReAct agent can sometimes enter the incorrect reasoning loop to answer the question.

In [16]:
from llama_index.agent import ReActAgent
from llama_index.llms import OpenAI

In [17]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
# llm = OpenAI(model="gpt-4-0613")
base_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [18]:
# gpt-3.5 generally gives the right response here
response = base_agent.chat(
    "Analyze Uber revenue growth over the last few quarters"
)
print(str(response))

Thought: To analyze Uber's revenue growth over the last few quarters, I can use the tools that provide information about Uber's quarterly financials.
Action: march_2022
Action Input: {'input': 'revenue'}
Observation: Revenue for the company primarily comes from fees paid by Drivers and Merchants for the use of their platform. They act as an agent in these transactions by connecting consumers to Drivers and Merchants to facilitate transportation services. Revenue is recognized when a trip is complete. In certain markets where the company is responsible for mobility services, revenue from end-users is presented on a gross basis, while payments to Drivers for mobility services are recognized in cost of revenue. Additionally, the company derives revenue from Delivery transactions and Freight transportation services.
Thought: The information provided gives an overview of Uber's revenue sources and recognition policies. However, it does not provide specific revenue growth figures over the la

wandb: Logged trace tree to W&B.


Thought: The information provided in September 2022 is similar to the previous observations and does not provide specific revenue growth figures over the last few quarters. It seems that the tools available do not provide the necessary information to analyze Uber's revenue growth.
Answer: Sorry, I cannot answer your query.
Sorry, I cannot answer your query.


In [19]:
# gpt-3.5 doesn't give the right response - it doesn't first search for the quarter with the highest revenue growth
response = base_agent.chat(
    "Can you tell me about the risk factors in the quarter with the highest"
    " revenue growth?"
)
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'risk factors'}
Observation: The risk factors mentioned in the context include:
1. Economic, social, weather, and regulatory conditions or other circumstances, including COVID-19, that may negatively affect operations.
2. Failure to offer autonomous vehicle technologies or their inferior performance compared to competitors.
3. Difficulty in retaining and attracting high-quality personnel.
4. Security or data privacy breaches or unauthorized access to proprietary or confidential data.
5. Cyberattacks that could harm reputation, business, and operating results.
6. Climate change risks, including physical and transitional risks, that may adversely impact the business.
7. Dependence on third parties for distributing the platform and providing software.
8. Need for additional capital to support business growth.
9. Risks associated with identifying, acquiring, and integrating suitable bus

wandb: Logged trace tree to W&B.


Thought: I have the information about the risk factors in the quarter with the highest revenue growth.
Answer: The risk factors mentioned in the quarter with the highest revenue growth include economic, social, weather, and regulatory conditions, failure to offer autonomous vehicle technologies, difficulty in retaining high-quality personnel, security or data privacy breaches, cyberattacks, climate change risks, dependence on third parties, need for additional capital, risks associated with acquiring and integrating businesses, legal and regulatory risks, extensive government regulation, risks related to data collection and processing, intellectual property protection, volatility in stock price, impact of the COVID-19 pandemic, legal proceedings and investigations, risks related to driver classification, competition in the mobility industry, lowering fares and offering incentives, significant losses and increased operating expenses, difficulty in attracting platform users, importance o

## Generate Training/Eval Questions

Generate a synthetic dataset of questions to ask. To do this, we generate an initial set of questions over a "base" document (the March 2022 10Q), and then we use an LLM to generate variations of that question that can apply across multiple quarters. This allows us to more deeply stress-test the LLM reasoning capabilities.


In [20]:
from llama_index.evaluation import DatasetGenerator

In [21]:
base_question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup a quiz/examination."
    " Using the provided context from the Uber March 10Q filing, formulate a"
    " single question that captures an important fact from the context."
    " context. Restrict the question to the context information provided."
)

dataset_generator = DatasetGenerator.from_documents(
    march_docs,
    question_gen_query=base_question_gen_query,
    service_context=gpt_35_context,
)

/usr/local/lib/python3.10/dist-packages/llama_index/evaluation/dataset_generation.py:184: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [22]:
import nest_asyncio
nest_asyncio.apply()

In [23]:
questions = dataset_generator.generate_questions_from_nodes(num=20)

wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.


Failed to log trace tree to W&B: list index out of range
Failed to log trace tree to W&B: list index out of range
Failed to log trace tree to W&B: list index out of range
Failed to log trace tree to W&B: list index out of range
Failed to log trace tree to W&B: list index out of range
Failed to log trace tree to W&B: list index out of range
Failed to log trace tree to W&B: list index out of range


wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.
wandb: Logged trace tree to W&B.


Failed to log trace tree to W&B: list index out of range


wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 125004 bytes
wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 249932 bytes
wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 499788 bytes
wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 999500 bytes
wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 1998924 bytes
wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 3997772 bytes
wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 7995468 bytes
wandb: Logged trace tree to W&B.
wandb: WARNING Serializing object of type str that is 15990860 bytes
wandb: Logged trace tree to W&B.
/usr/local/lib/python3.10/dist-packages/llama_index/evaluation/dataset_generation.py:279: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Depr

In [24]:
questions

["What is the address of Uber Technologies, Inc.'s principal executive offices?",
 'What is the last accessed date of the Uber March 10Q filing?',
 'Question: What is the primary purpose of the forward-looking statements mentioned in the Uber March 10Q filing?',
 'Question: What is the last accessed date of the Uber March 10Q filing?',
 'Question: What was the total amount of cash and cash equivalents held by Uber Technologies, Inc. as of March 31, 2022?',
 "What was Uber's net loss attributable to Uber Technologies, Inc. for the three months ended March 31, 2022?",
 'What was the net loss including non-controlling interests for Uber in the first quarter of 2022?',
 'Question: What was the total equity balance of Uber Technologies, Inc. as of March 31, 2021, according to the provided context information?',
 'What was the net income (loss) for Uber Technologies, Inc. in the first quarter of 2022?',
 'What was the net loss including non-controlling interests for Uber in the first quarter

In [ ]:
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate


vary_question_tmpl = """\
You are a financial assistant. Given a question over a 2023 Uber 10Q filing, your goal
is to generate up to {num_vary} variations of that question that might span multiple 10Q's.

This can include compare/contrasting different 10Qs, replacing the current quarter with
another quarter, or generating questions that can only be answered over multiple quarters (be creative!)

You are given a valid set of 10Q filings. Please only generate question variations that can be
answered in that set.

For example:
Base Question: What was the free cash flow of Uber in March 2023?
Valid 10Qs: [March 2023, June 2023, September 2023]
Question Variations:
What was the free cash flow of Uber in June 2023?
Can you compare/contrast the free cash flow of Uber in June/September 2023 and offer explanations for the change?
Did the free cash flow of Uber increase of decrease in 2023?

Now let's give it a shot!

Base Question: {base_question}
Valid 10Qs: {valid_10qs}
Question Variations:
"""


def gen_question_variations(base_questions, num_vary=3):
    """Generate question variations."""

    VALID_10Q_STR = "[March 2022, June 2022, September 2022]"

    llm = OpenAI(model="gpt-4")
    prompt_tmpl = PromptTemplate(vary_question_tmpl)

    new_questions = []
    for idx, question in enumerate(base_questions):
        new_questions.append(question)
        response = llm.complete(
            prompt_tmpl.format(
                num_vary=num_vary,
                base_question=question,
                valid_10qs=VALID_10Q_STR,
            )
        )
        # parse into newlines
        raw_lines = str(response).split("\n")
        cur_new_questions = [l for l in raw_lines if l != ""]
        print(f"[{idx}] Original Question: {question}")
        print(f"[{idx}] Generated Question Variations: {cur_new_questions}")
        new_questions.extend(cur_new_questions)

    return new_questions


def save_questions(questions, path):
    with open(path, "w") as f:
        for question in questions:
            f.write(question + "\n")


def load_questions(path):
    questions = []
    with open(path, "r") as f:
        for line in f:
            questions.append(line.strip())
    return questions

In [27]:
new_questions = gen_question_variations(questions)

[0] Original Question: What is the address of Uber Technologies, Inc.'s principal executive offices?
[0] Generated Question Variations: ["Has the address of Uber Technologies, Inc.'s principal executive offices changed between March and September 2022?", "What was the address of Uber Technologies, Inc.'s principal executive offices in June 2022?", "Can you provide the addresses of Uber Technologies, Inc.'s principal executive offices as listed in the March, June, and September 2022 10Q filings?"]
[1] Original Question: What is the last accessed date of the Uber March 10Q filing?
[1] Generated Question Variations: ['What is the last accessed date of the Uber June 10Q filing?', 'Can you compare the last accessed dates of the Uber 10Q filings for March, June, and September 2022?', 'What is the average time between the last accessed dates of the Uber 10Q filings in 2022?']
[2] Original Question: Question: What is the primary purpose of the forward-looking statements mentioned in the Uber M

In [28]:
len(new_questions)

80

In [29]:
train_questions, eval_questions = new_questions[:60], new_questions[60:]

In [30]:
#TODO: wandb save
save_questions(train_questions, "train_questions_10q.txt")
save_questions(eval_questions, "eval_questions_10q.txt")

In [31]:
train_questions = load_questions("train_questions_10q.txt")
eval_questions = load_questions("eval_questions_10q.txt")

## Use GPT-4 to Log Input/Output Pairs

We run the train questions through a GPT-4 powered ReAct agent to collect prompt outputs.

Every prompt call to the LLM is logged as an input/output pair. Since the ReAct loop can call the LLM multiple times, this means that multiple input/output pairs may be logged per user query.

Our `OpenAIFineTuningHandler` automatically collects prompt input/outputs when agent queries are run. This dataset can then be saved, in a dataset format `.jsonl` that you can directly feed to the OpenAI Finetuning endpoints.

In [32]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager
from llama_index.agent import ReActAgent

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [33]:
llm = OpenAI(model="gpt-4-0613")
gpt4_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    callback_manager=callback_manager,
    verbose=True,
)

In [34]:
for idx, question in enumerate(train_questions):
    print(f"[{idx}] Question: {question}")
    response = gpt4_agent.query(question)
    print(f"[{idx}] Agent Response: {str(response)}")

[0] Question: What is the address of Uber Technologies, Inc.'s principal executive offices?
Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices of Uber Technologies, Inc. are not explicitly mentioned in the provided context information.
Thought: The first tool did not provide the required information. I should try the other tools.
Action: june_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices of Uber Technologies, Inc. are not explicitly mentioned in the provided context information.
Thought: The second tool also did not provide the required information. I should try the last tool.
Action: sept_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices refer to the main offices or headquarters of a company where the top executives, such as the CEO and other ke

wandb: Logged trace tree to W&B.


Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query.
[0] Agent Response: Sorry, I cannot answer your query.
[1] Question: Has the address of Uber Technologies, Inc.'s principal executive offices changed between March and September 2022?
Thought: I need to use the tools to get the information about Uber's principal executive offices for March and September 2022.
Action: march_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices of Uber Technologies, Inc. are not explicitly mentioned in the provided context information.
Thought: I didn't get the information about Uber's principal executive offices for March 2022. I will now try to get the information for September 2022.
Action: sept_2022
Action Input: {'input': 'principal executive offices'}
Observation: The principal executive offices refer to the main offices or headquarters of a company where the top executives, such as the CEO and

wandb: Logged trace tree to W&B.


Thought: I cannot answer the question with the provided tools.
Answer: Sorry, I cannot answer your query. The tools provided do not contain specific information about the location of Uber Technologies, Inc.'s principal executive offices.
[1] Agent Response: Sorry, I cannot answer your query. The tools provided do not contain specific information about the location of Uber Technologies, Inc.'s principal executive offices.
[2] Question: What was the address of Uber Technologies, Inc.'s principal executive offices in June 2022?
Thought: I need to use the tool that provides information about Uber's financials for the quarter ending in June 2022 to find the address of Uber's principal executive offices during that time.
Action: june_2022
Action Input: {'input': 'principal executive offices address'}
Observation: I'm sorry, but the given context information does not provide the principal executive offices address for Uber Technologies, Inc.


...

wandb: Logged trace tree to W&B.


Thought: I can answer without using any more tools.
Answer: The total equity balance of Uber Technologies, Inc. as of March 31, 2022, was $9,613 million.
[29] Agent Response: The total equity balance of Uber Technologies, Inc. as of March 31, 2022, was $9,613 million.
[30] Question: Can you compare and contrast the total equity balance of Uber Technologies, Inc. in March 2022 and June 2022 and provide possible reasons for any changes?
Thought: I need to use the tools to get the total equity balance of Uber Technologies, Inc. for March 2022 and June 2022.
Action: march_2022
Action Input: {'input': 'total equity balance'}
Observation: The total equity balance as of March 31, 2022, is $9,613 million.
Thought: Now I need to use the tool to get the total equity balance of Uber Technologies, Inc. for June 2022.
Action: june_2022
Action Input: {'input': 'total equity balance'}
Observation: The total equity balance as of June 30, 2022, is $7,369 million.


wandb: Logged trace tree to W&B.


Thought: The total equity balance of Uber Technologies, Inc. decreased from $9,613 million in March 2022 to $7,369 million in June 2022. This indicates a decrease in equity. The decrease in equity could be due to a variety of factors such as losses incurred by the company, payment of dividends, or buyback of shares. However, without more specific financial details, it's not possible to definitively state the cause of the decrease.
Answer: The total equity balance of Uber Technologies, Inc. decreased from $9,613 million in March 2022 to $7,369 million in June 2022. This could be due to a variety of factors such as losses incurred by the company, payment of dividends, or buyback of shares. However, without more specific financial details, it's not possible to definitively state the cause of the decrease.
[30] Agent Response: The total equity balance of Uber Technologies, Inc. decreased from $9,613 million in March 2022 to $7,369 million in June 2022. This could be due to a variety of fac

KeyboardInterrupt: 

In [35]:
# save events
finetuning_handler.save_finetuning_events("finetuning_events_10q.jsonl")

Wrote 93 examples to finetuning_events_10q.jsonl


## Create `OpenAIFinetuneEngine`

We create an `OpenAIFinetuneEngine`: the finetune engine will launch a finetuning job, and returning an LLM model that you can directly plugin to the rest of LlamaIndex workflows.

In [36]:
dataset_path = "finetuning_events_10q.jsonl"

In [37]:
from wandb.integration.openai.fine_tuning import WandbLogger

In [38]:
def openai_validate_data(dataset_path):
    data_path = dataset_path

    # Load dataset
    with open(data_path) as f:
        dataset = [json.loads(line) for line in f]

    # We can inspect the data quickly by checking the number of examples and the first item

    # Initial dataset stats
    print("Num examples:", len(dataset))
    print("First example:")
    for message in dataset[0]["messages"]:
        print(message)

    # Now that we have a sense of the data, we need to go through all the different examples and check to make sure the formatting is correct and matches the Chat completions message structure

    # Format error checks
    format_errors = defaultdict(int)

    for ex in dataset:
        if not isinstance(ex, dict):
            format_errors["data_type"] += 1
            continue

        messages = ex.get("messages", None)
        if not messages:
            format_errors["missing_messages_list"] += 1
            continue

        for message in messages:
            if "role" not in message or "content" not in message:
                format_errors["message_missing_key"] += 1

            if any(k not in ("role", "content", "name") for k in message):
                format_errors["message_unrecognized_key"] += 1

            if message.get("role", None) not in ("system", "user", "assistant"):
                format_errors["unrecognized_role"] += 1

            content = message.get("content", None)
            if not content or not isinstance(content, str):
                format_errors["missing_content"] += 1

        if not any(message.get("role", None) == "assistant" for message in messages):
            format_errors["example_missing_assistant_message"] += 1

    if format_errors:
        print("Found errors:")
        for k, v in format_errors.items():
            print(f"{k}: {v}")
    else:
        print("No errors found")

    # Beyond the structure of the message, we also need to ensure that the length does not exceed the 4096 token limit.

    # Token counting functions
    encoding = tiktoken.get_encoding("cl100k_base")

    # not exact!
    # simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
    def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
        num_tokens = 0
        for message in messages:
            num_tokens += tokens_per_message
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":
                    num_tokens += tokens_per_name
        num_tokens += 3
        return num_tokens

    def num_assistant_tokens_from_messages(messages):
        num_tokens = 0
        for message in messages:
            if message["role"] == "assistant":
                num_tokens += len(encoding.encode(message["content"]))
        return num_tokens

    def print_distribution(values, name):
        print(f"\n#### Distribution of {name}:")
        print(f"min / max: {min(values)}, {max(values)}")
        print(f"mean / median: {np.mean(values)}, {np.median(values)}")
        print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

    # Last, we can look at the results of the different formatting operations before proceeding with creating a fine-tuning job:

    # Warnings and tokens counts
    n_missing_system = 0
    n_missing_user = 0
    n_messages = []
    convo_lens = []
    assistant_message_lens = []

    for ex in dataset:
        messages = ex["messages"]
        if not any(message["role"] == "system" for message in messages):
            n_missing_system += 1
        if not any(message["role"] == "user" for message in messages):
            n_missing_user += 1
        n_messages.append(len(messages))
        convo_lens.append(num_tokens_from_messages(messages))
        assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

    print("Num examples missing system message:", n_missing_system)
    print("Num examples missing user message:", n_missing_user)
    print_distribution(n_messages, "num_messages_per_example")
    print_distribution(convo_lens, "num_total_tokens_per_example")
    print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
    n_too_long = sum(l > 4096 for l in convo_lens)
    print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

    # Pricing and default n_epochs estimate
    MAX_TOKENS_PER_EXAMPLE = 4096

    MIN_TARGET_EXAMPLES = 100
    MAX_TARGET_EXAMPLES = 25000
    TARGET_EPOCHS = 3
    MIN_EPOCHS = 1
    MAX_EPOCHS = 25

    n_epochs = TARGET_EPOCHS
    n_train_examples = len(dataset)
    if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
        n_epochs = min(MAX_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
    elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
        n_epochs = max(MIN_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

    n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
    print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
    print(f"By default, you'll train for {n_epochs} epochs on this dataset")
    print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
    print("See pricing page to estimate total costs")

In [39]:
openai_validate_data(dataset_path)

Num examples: 93
First example:
{'role': 'system', 'content': '\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: march_2022\nTool Description: Provides information about Uber quarterly financials ending March 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: june_2022\nTool Description: Provides information about Uber quarterly financials ending June 2022\nTool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}\n\n> Tool Name: sept_2022\nToo

In [40]:
from openai import OpenAI as SyncOpenAI

In [41]:
oai_client = SyncOpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [42]:
openai_train_file_info = oai_client.files.create(
    file=open(dataset_path, "rb"), purpose="fine-tune"
)

In [43]:
model = 'gpt-3.5-turbo'
n_epochs = 3

In [44]:
openai_ft_job_info = oai_client.fine_tuning.jobs.create(
    training_file=openai_train_file_info.id,
    model=model,

    hyperparameters={"n_epochs": n_epochs},
    # validation_file=openai_valid_file_info.id
)

ft_job_id = openai_ft_job_info.id

In [45]:
WandbLogger.sync(fine_tune_job_id=ft_job_id, openai_client=oai_client)

wandb: Retrieving fine-tune job...
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Waiting for the OpenAI fine-tuning job to be finished...
wandb: Fine-tuning finished, logging metrics, model metadata, and more to W&B
wandb: ERROR File file-wf9PkgvLLAotpCBLSTn8Q9f2 could not be read as a valid JSON file
wandb: Waiting for artifact train-file-wf9PkgvLLAotpCBLSTn8Q9f2 to be committed...

wandb: Committed artifact keloasuzu/llama_index-react-agent-finetuning/train-file-wf9PkgvLLAotpCBLSTn8Q9f2:v0


'🎉 wandb sync completed successfully'

In [46]:
openai_ft_job_info

FineTuningJob(id='ftjob-r4jvGFFLrOmqibW5x1nmhf5S', created_at=1706227231, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-T17XYrfw4A1rxbU3EiTXGQpU', result_files=[], status='validating_files', trained_tokens=None, training_file='file-wf9PkgvLLAotpCBLSTn8Q9f2', validation_file=None)

In [47]:
state = oai_client.fine_tuning.jobs.retrieve(fine_tuning_job_id=ft_job_id)
state.status, state.trained_tokens, state.finished_at, state.fine_tuned_model

('succeeded', 203964, 1706228025, 'ft:gpt-3.5-turbo-0613:personal::8l48oaTz')

In [48]:
ft_llm = OpenAI(model=state.fine_tuned_model)

## Run Some Queries! (Compare Finetuned Agent vs. Base Agent)

We run some sample queries from the evaluation dataset over both our finetuned agent as well as the base agent.

We qualitatively look at their abilities to perform chain of thought prompting in order to arrive at the right answer.

**NOTE**: There's a big TODO to setup quantitative metrics so we can more rigorously evaluate the quality of any agent over an evaluation dataset!

In [49]:
# Option 1: pass in ft_llm directly into ServiceContext
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
)

In [51]:
ft_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=ft_llm,
    callback_manager=callback_manager,
    verbose=True,
)

In [52]:
eval_questions = []
with open("eval_questions_10q.txt", "r") as f:
    for line in f:
        eval_questions.append(line.strip())

In [53]:
# try a sample question
qidx = 0
print(eval_questions[qidx])

Question: What is the total fair value of financial assets measured at Level 1 as of March 31, 2022, according to the Uber March 10Q filing?


In [54]:
base_response = base_agent.query(eval_questions[qidx])
print(str(base_response))

Thought: I need to use a tool to help me answer the question.
Action: march_2022
Action Input: {'input': 'total fair value of financial assets measured at Level 1'}
Observation: The total fair value of financial assets measured at Level 1 is $5,811 million.
Thought: I have the information needed to answer the question.
Answer: The total fair value of financial assets measured at Level 1 as of March 31, 2022, according to the Uber March 10Q filing, is $5,811 million.


wandb: Logged trace tree to W&B.


The total fair value of financial assets measured at Level 1 as of March 31, 2022, according to the Uber March 10Q filing, is $5,811 million.


In [55]:
ft_response = ft_agent.query(eval_questions[qidx])
print(str(ft_response))

Thought: I need to use the march_2022 tool to get the information about the Uber March 10Q filing.
Action: march_2022
Action Input: {'input': 'total fair value of financial assets measured at Level 1'}
Observation: The total fair value of financial assets measured at Level 1 is $5,811 million.


wandb: Logged trace tree to W&B.


Thought: I can answer without using any more tools.
Answer: The total fair value of financial assets measured at Level 1 as of March 31, 2022, according to the Uber March 10Q filing, is $5,811 million.
The total fair value of financial assets measured at Level 1 as of March 31, 2022, according to the Uber March 10Q filing, is $5,811 million.


In [56]:
# try the original question that failed
test_q = (
    "Can you tell me about the risk factors in the quarter with the highest"
    " revenue growth?"
)
base_response = base_agent.query(test_q)
print(str(base_response))

Thought: To answer this question, I need to find the quarter with the highest revenue growth and then identify the risk factors associated with that quarter.

Thought: I can use the financial tools for each quarter to compare the revenue growth and identify the quarter with the highest growth.
Action: march_2022
Action Input: {'input': 'revenue_growth'}
Observation: The revenue growth for the company in the three months ended March 31, 2022, compared to the same period in 2021, was 136%.
Thought: The revenue growth for the quarter ending March 2022 was 136%. Now I need to find the risk factors associated with this quarter.
Action: march_2022
Action Input: {'input': 'risk_factors'}
Observation: The COVID-19 pandemic and the impact of actions to mitigate the pandemic have adversely affected and may continue to adversely affect parts of our business. Our business would be adversely affected if Drivers were classified as employees, workers or quasi-employees instead of independent contract

wandb: Logged trace tree to W&B.


Thought: I have identified the risk factors associated with the quarter ending March 2022. Here are some of the key risk factors:

1. The adverse impact of the COVID-19 pandemic and actions taken to mitigate it.
2. The potential reclassification of drivers as employees instead of independent contractors.
3. Intense competition in the mobility, delivery, and logistics industries.
4. The need to lower fares or service fees and offer driver incentives and consumer discounts to remain competitive.
5. Historical losses and the uncertainty of achieving profitability.
6. The importance of attracting and maintaining a critical mass of drivers, consumers, merchants, shippers, and carriers.
7. The significance of maintaining and enhancing the brand and reputation.
8. Operational, compliance, and cultural challenges associated with the company's workplace culture.
9. The need to optimize organizational structure and effectively manage growth.
10. Safety incidents that may harm the ability to attr

In [57]:
# NOTE: this successfully looks at each quarter for revenue growth but still falls behind GPT-4
ft_response = ft_agent.query(test_q)
print(str(ft_response))

Thought: I need to find the quarter with the highest revenue growth first.
Action: march_2022
Action Input: {'input': 'revenue growth'}
Observation: The revenue growth for the company in the three months ended March 31, 2022, compared to the same period in 2021, was 136%. This increase in revenue was primarily driven by an increase in Gross Bookings of 35%, or 39% on a constant currency basis. The growth in Gross Bookings was mainly due to increases in Mobility Trip volumes as the business recovers from the impacts of COVID-19 and a $1.5 billion increase in Freight revenue resulting from the acquisition of Transplace. Additionally, there was a $304 million increase in Delivery revenue and a $200 million increase in Mobility revenue due to business model changes in the UK.
Thought: Now that I have the quarter with the highest revenue growth (March 2022), I need to find the risk factors for that quarter.
Action: march_2022
Action Input: {'input': 'risk factors'}
Observation: The risk fac

wandb: Logged trace tree to W&B.


Thought: I can answer without using any more tools.
Answer: The risk factors in the quarter with the highest revenue growth (March 2022) for Uber include economic, social, weather, and regulatory conditions, including the ongoing impact of COVID-19. Other risk factors include the failure to offer competitive autonomous vehicle technologies, difficulty in retaining high-quality personnel, security and data privacy breaches, cyberattacks, climate change risks, dependence on third parties for distribution and software, the need for additional capital, legal and regulatory risks, risks related to data collection and processing, risks related to intellectual property, volatility in the stock market, legal proceedings and investigations, competition in the mobility, delivery, and logistics industries, incurred losses and expected increase in operating expenses, difficulty in attracting and maintaining platform users, the importance of maintaining brand and reputation, operational and complia

**Observations**: The finetuned model does much better than the base model in terms of reasoning about the current sequence of steps. It passes more detailed answers to the downstream tools and is more capable of refining its approach when initial queries don't work. This applies even if the answer isn't actually found within the context (which is a function of our automatic dataset generation capabilities).